In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

pd.set_option('display.max_columns', None)

## Colorado traffic data: clean up and apply VMT breakdown from CDPHE

In [37]:
#CDPHE 2017 VMT breakdown by functional system classification and rural/urban. The breakdown is for weekday travel. 
cdphe = pd.read_excel('../Transportation/Colorado/CDPHE_VMTbreakdown_byHPMStype_byRoadType.xlsx', skiprows=1)
cdphe = cdphe.groupby(['Rural?','FC']).mean().reset_index()
cdphe.drop(columns='NAA?', inplace=True)
cdphe['func_R/U'] = cdphe['Rural?']+'_'+cdphe['FC'].astype(str)
cdphe.set_index('func_R/U', inplace=True)

In [38]:
vehicle_cols = ['Motorcycle', 'Passenger cars','Light trucks', 'Buses', 'SU trucks', 'CU trucks']

In [39]:
cotraffic = pd.read_csv('../Transportation/Colorado/CO_aadt_2018.csv')
cotraffic.loc[cotraffic['urban_code']==99999,'Rural?'] = 'R'
cotraffic.loc[cotraffic['urban_code']!=99999,'Rural?'] = 'U'
cotraffic['func_R/U'] = cotraffic['Rural?']+'_'+cotraffic['f_system'].astype(str)

In [40]:
allna_inds = cotraffic['aadt'].isna()& cotraffic['aadt_combination'].isna()& cotraffic['aadt_single_unit'].isna()
cotraffic.drop(cotraffic[allna_inds].index, inplace=True)

In [41]:
truckna_inds = cotraffic['aadt_combination'].isna() & cotraffic['aadt_single_unit'].isna()

In [7]:
for col in vehicle_cols:
    cotraffic['aadt_'+col] = 0
    
cotraffic['aadt_CU trucks'] = cotraffic['aadt_combination']
cotraffic['aadt_SU trucks'] = cotraffic['aadt_single_unit']
cotraffic.drop(columns=['aadt_combination','aadt_single_unit'],inplace=True)

for roadtype in cotraffic['func_R/U'].unique():
    roadtype_inds = cotraffic['func_R/U']==roadtype
    breakdown = cdphe.loc[roadtype]
    road_truckna_inds = roadtype_inds & truckna_inds
    for col in vehicle_cols:
        aadt_frac = breakdown[col]
        cotraffic.loc[road_truckna_inds,'aadt_'+col] = aadt_frac * cotraffic[road_truckna_inds]['aadt']

In [8]:
light_cols = ['Motorcycle', 'Passenger cars','Light trucks', 'Buses']
normalizer = cdphe[light_cols].sum(axis=1)
leftover_aadt = cotraffic['aadt'] - cotraffic['aadt_CU trucks'] - cotraffic['aadt_SU trucks']
for roadtype in cotraffic['func_R/U'].unique():
    roadtype_inds = cotraffic['func_R/U']==roadtype
    renormalized_breakdown = cdphe.loc[roadtype, light_cols] / normalizer.loc[roadtype]
    road_with_truck_inds = roadtype_inds & ~truckna_inds
    for col in light_cols:
        cotraffic.loc[road_with_truck_inds,'aadt_'+col] = leftover_aadt[road_with_truck_inds]*renormalized_breakdown[col]

In [9]:
cotraffic.to_csv('../Transportation/Colorado/co_aadt_2018_updated.csv')

## Nevada traffic data: clean up and granularize

In [10]:
#HPMS 2018 VM-4 VMT distribution by road category and rural/urban.
hpms_nv = pd.read_csv('../Transportation/HPMS_VM4_2018_NevadaVMTbreakdown_byRoadType_byRuralUrban.csv')
hpms_nv.drop(columns={'Total','Unnamed: 10'}, inplace = True)
hpms_nv['roadcategory_R/U'] = hpms_nv['Road Type']+'_'+hpms_nv['Rural?'].astype(str)
hpms_nv[vehicle_cols] = hpms_nv[vehicle_cols]/100
hpms_nv.set_index('roadcategory_R/U', inplace=True)

In [11]:
mapping = {1: 'Interstate',
           2: 'Other Arterials',
           3: 'Other Arterials',
           4: 'Other Arterials',
           5: 'Other',
           6: 'Other',
           7: 'Other'
           }

In [12]:
nvtraffic = pd.read_csv('../Transportation/Nevada/NV_HPMS_aadt_2018.csv')

In [13]:
for fsystem in nvtraffic['f_system'].unique():
    road_category = mapping[fsystem]
    fsystem_inds = nvtraffic['f_system'] == fsystem
    nvtraffic.loc[fsystem_inds,'road_category'] = road_category

In [14]:
nvtraffic.loc[nvtraffic['urban_code']==99999,'Rural?'] = 'R'
nvtraffic.loc[nvtraffic['urban_code']!=99999,'Rural?'] = 'U'
nvtraffic['roadcategory_R/U'] = nvtraffic['road_category']+'_'+nvtraffic['Rural?'].astype(str)

In [15]:
allna_inds = nvtraffic['aadt'].isna()& nvtraffic['aadt_combination'].isna()& nvtraffic['aadt_single_unit'].isna()
nvtraffic.drop(nvtraffic[allna_inds].index, inplace=True)

In [16]:
combna_inds = nvtraffic['aadt_combination'].isna() 
singna_inds = nvtraffic['aadt_single_unit'].isna()
truckna_inds = combna_inds & singna_inds
#clark_inds = nvtraffic['county_code']== 3

In [17]:
for col in vehicle_cols:
    nvtraffic['aadt_'+col] = 0
    
nvtraffic['aadt_CU trucks'] = nvtraffic['aadt_combination']
nvtraffic['aadt_SU trucks'] = nvtraffic['aadt_single_unit']
nvtraffic.drop(columns=['aadt_combination','aadt_single_unit'],inplace=True)

for roadtype in nvtraffic['roadcategory_R/U'].unique():
    roadtype_inds = nvtraffic['roadcategory_R/U']==roadtype
    breakdown = hpms_nv.loc[roadtype]
    road_truckna_inds = roadtype_inds & truckna_inds
    for col in vehicle_cols:
        aadt_frac = breakdown[col]
        nvtraffic.loc[road_truckna_inds,'aadt_'+col] = aadt_frac * nvtraffic[road_truckna_inds]['aadt']

In [18]:
exceptcomb_cols_nv = ['Motorcycle', 'Passenger cars','Light trucks', 'Buses','SU trucks']
normalizer_exceptcomb = hpms_nv[exceptcomb_cols_nv].sum(axis=1)
leftover_aadt_nocomb_nv = nvtraffic['aadt'] - nvtraffic['aadt_CU trucks'] 
for roadtype in nvtraffic['roadcategory_R/U'].unique():
    roadtype_inds = nvtraffic['roadcategory_R/U']==roadtype
    renormalized_breakdown = hpms_nv.loc[roadtype, exceptcomb_cols_nv] / normalizer_exceptcomb.loc[roadtype]
    road_with_singna_inds = roadtype_inds & singna_inds & ~combna_inds
    for col in exceptcomb_cols_nv:
        nvtraffic.loc[road_with_singna_inds,'aadt_'+col] = leftover_aadt_nocomb_nv[road_with_singna_inds]*renormalized_breakdown[col]
        

In [19]:
light_cols_nv = ['Motorcycle', 'Passenger cars','Light trucks', 'Buses']
normalizer_light = hpms_nv[light_cols].sum(axis=1)
leftover_aadt_nv = nvtraffic['aadt'] - nvtraffic['aadt_CU trucks'] - nvtraffic['aadt_SU trucks']
for roadtype in nvtraffic['roadcategory_R/U'].unique():
    roadtype_inds = nvtraffic['roadcategory_R/U']==roadtype
    renormalized_breakdown_light = hpms_nv.loc[roadtype, light_cols_nv] / normalizer_light.loc[roadtype]
    road_with_truck_inds = roadtype_inds & ~truckna_inds
    for col in light_cols:
        nvtraffic.loc[road_with_truck_inds,'aadt_'+col] = leftover_aadt_nv[road_with_truck_inds]*renormalized_breakdown_light[col]
        

In [20]:
nvtraffic.to_csv('../Transportation/Nevada/nv_aadt_2018_updated.csv')

## New Mexico traffic data: clean up and granularize

In [21]:
nmtraffic = pd.read_csv('../Transportation/New Mexico/NM_VMTbyRoadSegment_byVehicleType_2018_April2020.csv')
nmtraffic.loc[nmtraffic['FUNCCODE'].isna(),'FUNCCODE'] = 7.0
nmtraffic['FUNC_POP'] = nmtraffic['FUNCCODE'].astype(str)+'_'+ nmtraffic['POP_CODE']


In [22]:
vmt_cols = ['VMT_MC_Class 1',
       'VMT_PC_Class 2', 'VMT_LT_Class 3', 'VMT_Bus_Class 4', 'VMT_SU_Class 5',
       'VMT_SU_Class 6', 'VMT_SU_Class 7', 'VMT_CT_Class 8', 'VMT_CT_Class 9',
       'VMT_CT_Class 10', 'VMT_CT_Class 11', 'VMT_CT_Class 12',
       'VMT_CT_Class 13']

In [23]:
aadt_cols = ['AADT_MC_Class 1', 'AADT_PC_Class 2',
       'AADT_LT_Class 3', 'AADT_Bus_Class 4', 'AADT_SU_Class 5',
       'AADT_SU_Class 6', 'AADT_SU_Class 7', 'AADT_CT_Class 8',
       'AADT_CT_Class 9', 'AADT_CT_Class 10', 'AADT_CT_Class 11',
       'AADT_CT_Class 12', 'AADT_CT_Class 13']

In [24]:
missing_breakdown_inds = nmtraffic[aadt_cols].sum(axis=1) == 0
breakdown = nmtraffic[~missing_breakdown_inds]

In [25]:
grouped = breakdown.groupby(['FUNC_POP'])[vmt_cols].sum()
for col in grouped:
    grouped['frac_'+col] = 0

In [26]:
for funcpop in grouped.index:
    denom = grouped.loc[funcpop][vmt_cols].sum()
    for col in grouped[vmt_cols]:
        grouped.loc[funcpop,'frac_'+col] = grouped.loc[funcpop][col] / denom
        
grouped.drop(columns = vmt_cols,inplace=True)
grouped.columns = [column.replace('frac_VMT_','AADT_') for column in grouped]

In [27]:
for funcpop in grouped.index:
    funcpop_inds = nmtraffic['FUNC_POP'] == funcpop
    for col in aadt_cols:
        frac = grouped.loc[funcpop][col]
        nmtraffic.loc[missing_breakdown_inds & funcpop_inds,col] = frac * nmtraffic[missing_breakdown_inds & funcpop_inds]['AADT']
        
nmtraffic.drop(columns=vmt_cols+['FUNC_POP'], inplace=True)

In [28]:
nmtraffic.to_csv('../Transportation/New Mexico/nmdetail_aadt_2018_updated.csv')